In [1]:
from xai.dqn import DQN
from xai.stream import Stream
from xai.bytes import GigaBytes
     

dqn = DQN(translate=True, rotate=True, device="cpu", replay_buffer_size=66)     

In [2]:
dqn.train(
    total_time_steps=10000,
)

CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)


KeyboardInterrupt: 

In [ ]:
dqn.play(show=True, show_decode=True)

In [ ]:
import torch

X = torch.zeros((1,3,210,160)).float()

policy = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3,3), stride=(1,1)), # (102, 76)
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(5,5), stride=(2,2)), # (107, 76)
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(7,7), stride=(3,3)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(7,7), stride=(4,4)),
    torch.nn.ReLU(),
    )

policy(X).shape

In [ ]:
64*7*5